In [28]:
import pandas as pd
import numpy as np
import nltk as nt
nt.download('punkt')

rdata = pd.read_csv('Datafiniti_Hotel_Reviews.csv', header=0, sep=',', on_bad_lines='skip')
rtext, rtitle = rdata['reviews.text'].astype(str), rdata['reviews.title'].astype(str)

# remove "... More" from reviews (if it exists)
rtext = rtext.str.replace('... More', '', regex=False)

# tokenise
rtext_tk = rtext.apply(nt.word_tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\211612Y\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0    [Our, experience, at, Rancho, Valencia, was, a...
1    [Amazing, place, ., Everyone, was, extremely, ...
2    [We, booked, a, 3, night, stay, at, Rancho, Va...
3    [Currently, in, bed, writing, this, for, the, ...
4    [I, live, in, Md, and, the, Aloft, is, my, Hom...
Name: reviews.text, dtype: object
